In [1]:
import json
import math
import time
import secrets
import requests
import credentials
import numpy as np
from web3 import Web3
from datetime import datetime
from eth_account import Account

In [2]:
INFURA_KEY = credentials.infura_key
ETHERSCAN_KEY = credentials.etherscan_key

RPC = {
    'mainnet': f'https://mainnet.infura.io/v3/{INFURA_KEY}',
    'goreli': f'https://goerli.infura.io/v3/{INFURA_KEY}',
    'mainnet_ws': f'wss://mainnet.infura.io/ws/v3/{INFURA_KEY}',
    'goreli_ws': f'wss://goerli.infura.io/ws/v3/{INFURA_KEY}',
    'mainnet_mev': f'https://rpc.mevblocker.io'
}

# Mainnet MEV protector
provider = RPC['mainnet_mev']
w3 = Web3(Web3.HTTPProvider(provider, request_kwargs={'timeout': 60}))

In [3]:
def get_abi(contract_address, chain_id=1):
    #Get Contract ABI with Etherscan API
    api_key = ETHERSCAN_KEY
    if chain_id == 1:
        url = 'https://api.etherscan.io/api'
    elif chain_id == 5:
        url = 'https://api-goerli.etherscan.io/api'
    params = {
        'module': 'contract',
        'action': 'getabi',
        'address': contract_address,
        'apikey': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] != '1':
        print("Error: ", data['message'])
        return None
    else:
        abi = data['result']
        return abi

In [5]:
UNISWAP_V3_FACTORY = Web3.to_checksum_address("0x1F98431c8aD98523631AE4a59f267346ea31F984")
UNISWAP_V3_FACTORY_ABI = get_abi(UNISWAP_V3_FACTORY, chain_id=1)
UNISWAP_V2_FACTORY_CONTRACT = w3.eth.contract(address=UNISWAP_V3_FACTORY, abi=UNISWAP_V3_FACTORY_ABI)

In [13]:
USDT = Web3.to_checksum_address("0xdac17f958d2ee523a2206206994597c13d831ec7")
WETH = Web3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")
pool_address = UNISWAP_V2_FACTORY_CONTRACT.functions.getPool(WETH, USDT, 500).call()
pool_abi = get_abi(pool_address, chain_id=1)
pool_contract = w3.eth.contract(address=pool_address, abi=pool_abi)

In [15]:
pool_contract.all_functions()

[<Function burn(int24,int24,uint128)>,
 <Function collect(address,int24,int24,uint128,uint128)>,
 <Function collectProtocol(address,uint128,uint128)>,
 <Function factory()>,
 <Function fee()>,
 <Function feeGrowthGlobal0X128()>,
 <Function feeGrowthGlobal1X128()>,
 <Function flash(address,uint256,uint256,bytes)>,
 <Function increaseObservationCardinalityNext(uint16)>,
 <Function initialize(uint160)>,
 <Function liquidity()>,
 <Function maxLiquidityPerTick()>,
 <Function mint(address,int24,int24,uint128,bytes)>,
 <Function observations(uint256)>,
 <Function observe(uint32[])>,
 <Function positions(bytes32)>,
 <Function protocolFees()>,
 <Function setFeeProtocol(uint8,uint8)>,
 <Function slot0()>,
 <Function snapshotCumulativesInside(int24,int24)>,
 <Function swap(address,bool,int256,uint160,bytes)>,
 <Function tickBitmap(int16)>,
 <Function tickSpacing()>,
 <Function ticks(int24)>,
 <Function token0()>,
 <Function token1()>]